In [1]:
import pandas as pd
import numpy as np

In [2]:
store_status = pd.read_csv("../csv/store status.csv")
store_business_status = pd.read_csv("../csv/Menu hours.csv")
store_timezone = pd.read_csv("../csv/bq-results-20230125-202210-1674678181880.csv")

store_status.info()
store_business_status.info()
store_timezone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822080 entries, 0 to 1822079
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   store_id       int64 
 1   status         object
 2   timestamp_utc  object
dtypes: int64(1), object(2)
memory usage: 41.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86198 entries, 0 to 86197
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   store_id          86198 non-null  int64 
 1   day               86198 non-null  int64 
 2   start_time_local  86198 non-null  object
 3   end_time_local    86198 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13559 entries, 0 to 13558
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_id      13559 non-null  int64 
 1   timezone_str  13559 non-null 

In [3]:
store_id_1 = store_status.store_id.unique()
store_id_2 = store_business_status.store_id.unique()
store_id_3 = store_timezone.store_id.unique()

print(store_id_1.size)
print(store_id_2.size)
print(store_id_3.size)

14092
11116
13559


In [4]:
union_12 = np.union1d(store_id_1, store_id_2)
union_23 = np.union1d(store_id_2, store_id_3)
actual_store_ids = np.union1d(union_12, union_23)

actual_store_ids.size

14226

In [5]:
# locate missing store ids in the database.
# missing_id_1 = actual_store_ids[~np.isin(actual_store_ids, store_id_1)].tolist()
missing_id_2 = actual_store_ids[~np.isin(actual_store_ids, store_id_2)].tolist()
missing_id_3 = actual_store_ids[~np.isin(actual_store_ids, store_id_3)].tolist()

# print(len(missing_id_1))
print(len(missing_id_2))
print(len(missing_id_3))

134
3110
667


In [10]:
# generate dataframes
temp_df = None
store_timezone_append = None
store_business_status_append = None

for id in missing_id_3:
    temp_df = pd.DataFrame([[id, "America/Chicago"]], columns=["store_id", "timezone_str"])
    store_timezone_append = pd.concat([store_timezone_append, temp_df], ignore_index=True)
    
for id in missing_id_2:
    temp_df = pd.DataFrame([
        [id, 0, "00:00:00", "23:59:59"],
        [id, 1, "00:00:00", "23:59:59"],
        [id, 2, "00:00:00", "23:59:59"],
        [id, 3, "00:00:00", "23:59:59"],
        [id, 4, "00:00:00", "23:59:59"],
        [id, 5, "00:00:00", "23:59:59"],
        [id, 6, "00:00:00", "23:59:59"]
    ], columns=["store_id", "day", "start_time_local", "end_time_local"])
    store_business_status_append = pd.concat([store_business_status_append, temp_df], ignore_index=True)

store_timezone_append.to_csv("../csv/bq-results-20230125-202210-1674678181880.csv", mode="a", index=False, header=False)
store_business_status_append.to_csv("../csv/Menu hours.csv", mode="a", index=False, header=False)